**ROSBAG Cheat Sheet in Python**

There are two libraries that I found useful to open a rosbag file in a Jupyter notebook using Python: **rosbag** and **bagpy**.

To install them, the simplest way is to run 
```
pip3 install bagpy
```

**Note**: The python version I used is 3.8.8 (I have seen multiple issues of Python incompatibility but this worked)

In [17]:
import rosbag, bagpy, datetime
bagpy_bag = bagpy.bagreader('LEAD_delayed_2019-09-25-19-00-01-filtered.bag')
bag = rosbag.Bag('LEAD_delayed_2019-09-25-19-00-01-filtered.bag')

[INFO]  Data folder LEAD_delayed_2019-09-25-19-00-01-filtered already exists. Not creating.


Both opened up the file but in different forms. I find they each had their advantages.

**List of Topics** <br>
Reading topics from a bagreader file was cleaner and easily accessible compared to using a Bag file. 

In [2]:
#Topics using bagpy
print(bagpy_bag.topic_table)

                                               Topics  \
0               /delayed/artemisJr/centre/camera_info   
1   /delayed/artemisJr/centre/image_rect_color/com...   
2           /delayed/artemisJr/imageData/centrePtzCam   
3                 /delayed/artemisJr/left/camera_info   
4   /delayed/artemisJr/left/image_rect_color/compr...   
5   /delayed/artemisJr/right/image_rect_color/comp...   
6      /delayed/evo/left_polled/image_rect/compressed   
7     /delayed/evo/right_polled/image_rect/compressed   
8                     /delayed/lowrate_scanner/points   
9                                  /delayed/tf_static   
10  /delayed/trt/ws_left_polled/image_rect_color/c...   
11  /delayed/trt/ws_right_polled/image_rect_color/...   
12                     /delayed/trt_localization/pose   
13              /delayed/trt_stitcher/pano/compressed   

                          Types  Message Count  Frequency  
0        sensor_msgs/CameraInfo           1338   0.999854  
1   sensor_msgs/Compress

In [3]:
#Topics using rosbag
topics = bag.get_type_and_topic_info()[1].keys()
types = []
for val in bag.get_type_and_topic_info()[1].values():
    types.append(val[0])
print(topics)

dict_keys(['/delayed/artemisJr/centre/camera_info', '/delayed/artemisJr/centre/image_rect_color/compressed', '/delayed/artemisJr/imageData/centrePtzCam', '/delayed/artemisJr/left/camera_info', '/delayed/artemisJr/left/image_rect_color/compressed', '/delayed/artemisJr/right/image_rect_color/compressed', '/delayed/evo/left_polled/image_rect/compressed', '/delayed/evo/right_polled/image_rect/compressed', '/delayed/lowrate_scanner/points', '/delayed/tf_static', '/delayed/trt/ws_left_polled/image_rect_color/compressed', '/delayed/trt/ws_right_polled/image_rect_color/compressed', '/delayed/trt_localization/pose', '/delayed/trt_stitcher/pano/compressed'])


**Extracting messages**

I was also able to easily extract csv files using bagpy but could not find a way to do the same using rosbag.

In [4]:
import pandas as pd
Camera_info_left_msg = bagpy_bag.message_by_topic('/delayed/artemisJr/left/camera_info')
df_CAMERA_LEFT = pd.read_csv(Camera_info_left_msg) #Left camera calibration

df_CAMERA_LEFT.head(5)

,Time,header.seq,header.stamp.secs,header.stamp.nsecs,header.frame_id,height,width,distortion_model,D_0,D_1,...,P_9,P_10,P_11,binning_x,binning_y,roi.x_offset,roi.y_offset,roi.height,roi.width,roi.do_rectify
0,1.569452e+09,1,1569452267,390683889,/axis_camera,480,640,plumb_bob,-0.229407,0.037254,...,0.0,1.0,0.0,0,0,0,0,0,0,False
1,1.569455e+09,2,1569454496,488482952,/axis_camera,480,640,plumb_bob,-0.229407,0.037254,...,0.0,1.0,0.0,0,0,0,0,0,0,False
2,1.569455e+09,3,1569455429,276626110,/axis_camera,480,640,plumb_bob,-0.229407,0.037254,...,0.0,1.0,0.0,0,0,0,0,0,0,False
3,1.569455e+09,4,1569455430,279575109,/axis_camera,480,640,plumb_bob,-0.229407,0.037254,...,0.0,1.0,0.0,0,0,0,0,0,0,False
4,1.569455e+09,5,1569455431,279694080,/axis_camera,480,640,plumb_bob,-0.229407,0.037254,...,0.0,1.0,0.0,0,0,0,0,0,0,False


However, rosbag proved useful when one needed to loop through messages in a bag, and access specific parts of messages. <br>
For example, if I want to get the timestamps of images with the topic */delayed/trt/ws_right_polled/image_rect_color/compressed*:

In [18]:
for message in bag:
    if message[0] == "/delayed/trt/ws_right_polled/image_rect_color/compressed": 
        timestamp = datetime.datetime.fromtimestamp(message[2].to_sec()).strftime("%Y_%m_%d-%H_%M_%S") 
        print(timestamp)

2019_09_25-16_49_34
2019_09_25-17_14_38
2019_09_25-17_18_56
2019_09_25-17_42_26
2019_09_25-18_22_15
2019_09_25-18_26_00
2019_09_25-18_28_20
2019_09_25-18_34_54
2019_09_25-19_03_30


You can typically access a message topic 
```
print(message[0])
```
>/delayed/trt_localization/pose

You can access the header and more (position, data, depending on the message type)
```
print(message[1])
```

>```
header: 
  seq: 17028
  stamp: 
    secs: 1569468346
    nsecs: 465568065
  frame_id: "map"
pose: 
  position: 
    x: -85.49633974238243
    y: 7.322373946917995
    z: -0.36845247589603397
  orientation: 
    x: -0.002977009520887292
    y: 0.0028634341765548174
    z: 0.9999765338832188
    w: 0.005465330907904111


And finally, the timestamp of the message can be obtained
```
print(message[2])
```
> ```
1569468351487230074
```
